In [1]:
!pip install rdkit -q
!pip install torch_geometric -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.9/34.9 MB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 5.2 MB/s eta 0:00:00


In [2]:
import pandas as pd
from tqdm import tqdm

import numpy as np

from rdkit import Chem
from rdkit.Chem.rdmolops import GetAdjacencyMatrix

import torch
from torch_geometric.data import Data, DataLoader

import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
data = pd.read_excel('/content/19_35000.xlsx')

In [22]:
import math
transformed_data_log = data.IC50.apply(lambda x: math.log(x) * -1)

In [23]:
x_smiles = data.SMILES.to_numpy()[300:36000]
y = transformed_data_log.to_numpy()[300:36000]

In [6]:
def one_hot_encoding(x, permitted_list):

    if x not in permitted_list:
        x = permitted_list[-1]

    binary_encoding = [int(boolean_value) for boolean_value in list(map(lambda s: x == s, permitted_list))]

    return binary_encoding

## Признаки Атома:

Тип атома, число соседей-тяжелых атомов, формальный заряд, тип гибридизации, находится ли атом в кольце, является ли атом ароматическим, атомная масса, радиус Ван-дер-Ваальса и ковалентный радиус.

In [8]:
def get_atom_features(atom,
                      use_chirality = True,
                      hydrogens_implicit = True):

    permitted_list_of_atoms =  ['C','N','O','S','F','Si','P','Cl','Br','Mg','Na','Ca','Fe','As','Al','I', 'B','V','K','Tl','Yb','Sb','Sn','Ag','Pd','Co','Se','Ti','Zn', 'Li','Ge','Cu','Au','Ni','Cd','In','Mn','Zr','Cr','Pt','Hg','Pb','Unknown']

    if hydrogens_implicit == False:
        permitted_list_of_atoms = ['H'] + permitted_list_of_atoms


    atom_type_enc = one_hot_encoding(str(atom.GetSymbol()), permitted_list_of_atoms)

    n_heavy_neighbors_enc = one_hot_encoding(int(atom.GetDegree()), [0, 1, 2, 3, 4, "MoreThanFour"])

    formal_charge_enc = one_hot_encoding(int(atom.GetFormalCharge()), [-3, -2, -1, 0, 1, 2, 3, "Extreme"])

    hybridisation_type_enc = one_hot_encoding(str(atom.GetHybridization()), ["S", "SP", "SP2", "SP3", "SP3D", "SP3D2", "OTHER"])

    is_in_a_ring_enc = [int(atom.IsInRing())]

    is_aromatic_enc = [int(atom.GetIsAromatic())]

    atomic_mass_scaled = [float((atom.GetMass() - 10.812)/116.092)]

    vdw_radius_scaled = [float((Chem.GetPeriodicTable().GetRvdw(atom.GetAtomicNum()) - 1.5)/0.6)]

    covalent_radius_scaled = [float((Chem.GetPeriodicTable().GetRcovalent(atom.GetAtomicNum()) - 0.64)/0.76)]

    atom_feature_vector = atom_type_enc + n_heavy_neighbors_enc + formal_charge_enc + hybridisation_type_enc + is_in_a_ring_enc + is_aromatic_enc + atomic_mass_scaled + vdw_radius_scaled + covalent_radius_scaled

    if use_chirality == True:
        chirality_type_enc = one_hot_encoding(str(atom.GetChiralTag()), ["CHI_UNSPECIFIED", "CHI_TETRAHEDRAL_CW", "CHI_TETRAHEDRAL_CCW", "CHI_OTHER"])
        atom_feature_vector += chirality_type_enc

    if hydrogens_implicit == True:
        n_hydrogens_enc = one_hot_encoding(int(atom.GetTotalNumHs()), [0, 1, 2, 3, 4, "MoreThanFour"])
        atom_feature_vector += n_hydrogens_enc

    return np.array(atom_feature_vector)

## Признаки связи:
Тип связи, является ли связь сопряженной и находится ли связь в кольце

In [9]:
def get_bond_features(bond,
                      use_stereochemistry = True):

    permitted_list_of_bond_types = [Chem.rdchem.BondType.SINGLE, Chem.rdchem.BondType.DOUBLE, Chem.rdchem.BondType.TRIPLE, Chem.rdchem.BondType.AROMATIC]

    bond_type_enc = one_hot_encoding(bond.GetBondType(), permitted_list_of_bond_types)

    bond_is_conj_enc = [int(bond.GetIsConjugated())]

    bond_is_in_ring_enc = [int(bond.IsInRing())]

    bond_feature_vector = bond_type_enc + bond_is_conj_enc + bond_is_in_ring_enc

    if use_stereochemistry == True:
        stereo_type_enc = one_hot_encoding(str(bond.GetStereo()), ["STEREOZ", "STEREOE", "STEREOANY", "STEREONONE"])
        bond_feature_vector += stereo_type_enc

    return np.array(bond_feature_vector)

In [10]:
def create_pytorch_geometric_graph_data_list_from_smiles_and_labels(x_smiles, y):

    data_list = []

    for (smiles, y_val) in tqdm(zip(x_smiles, y)):

        mol = Chem.MolFromSmiles(smiles)

        n_nodes = mol.GetNumAtoms()
        n_edges = 2*mol.GetNumBonds()
        unrelated_smiles = "O=O"
        unrelated_mol = Chem.MolFromSmiles(unrelated_smiles)
        n_node_features = len(get_atom_features(unrelated_mol.GetAtomWithIdx(0)))
        n_edge_features = len(get_bond_features(unrelated_mol.GetBondBetweenAtoms(0,1)))

        X = np.zeros((n_nodes, n_node_features))

        for atom in mol.GetAtoms():
            X[atom.GetIdx(), :] = get_atom_features(atom)

        X = torch.tensor(X, dtype = torch.float)

        (rows, cols) = np.nonzero(GetAdjacencyMatrix(mol))
        torch_rows = torch.from_numpy(rows.astype(np.int64)).to(torch.long)
        torch_cols = torch.from_numpy(cols.astype(np.int64)).to(torch.long)
        E = torch.stack([torch_rows, torch_cols], dim = 0)

        EF = np.zeros((n_edges, n_edge_features))

        for (k, (i,j)) in enumerate(zip(rows, cols)):

            EF[k] = get_bond_features(mol.GetBondBetweenAtoms(int(i),int(j)))

        EF = torch.tensor(EF, dtype = torch.float)

        y_tensor = torch.tensor(np.array([y_val]), dtype = torch.float)

        data_list.append(Data(x = X, edge_index = E, edge_attr = EF, y = y_tensor))

    return data_list

In [24]:
data_list = create_pytorch_geometric_graph_data_list_from_smiles_and_labels(x_smiles, y)

35499it [01:37, 345.24it/s][20:45:07] Conflicting single bond directions around double bond at index 7.
[20:45:07]   BondStereo set to STEREONONE and single bond directions set to NONE.
35700it [01:37, 364.44it/s]


In [25]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data_list, y, test_size=0.2, random_state=42)

In [184]:
loader = DataLoader(dataset = X_train, batch_size = 1024)
test_loader = DataLoader(dataset = X_test, batch_size = 1024)

In [195]:
import torch
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, TopKPooling, global_mean_pool, BatchNorm
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
from torch_geometric.nn import GCNConv, GATConv, GraphConv, global_mean_pool, global_max_pool, TransformerConv


embedding_size = 32
dropout_rate = 0.4

class GCN(torch.nn.Module):
    def __init__(self):
        super(GCN, self).__init__()
        torch.manual_seed(42)

        # GCN layers
        self.initial_conv = TransformerConv(data_list[0].num_features, embedding_size)
        # self.conv1 = TransformerConv(embedding_size * 5, embedding_size , heads= 5)
        # self.conv2 = TransformerConv(embedding_size * 5, embedding_size, heads= 5)
        # self.bn1 = BatchNorm(embedding_size *5)
        # self.bn2 = BatchNorm(embedding_size * 5)
        # self.bn3 = BatchNorm(embedding_size * 5)

        # Additional Linear layers
        # self.fc1 = Linear(embedding_size, 128)
        # self.fc2 = Linear(128, 64)

        # Output layer
        self.out = Linear(64, 1)

    # def init_weights(self):
        # Initialization of GCN layers
        # self.initial_conv.reset_parameters()
        # self.conv1.reset_parameters()
        # self.conv2.reset_parameters()

        # # Initialization of BatchNorm layers
        # self.bn1.reset_parameters()
        # self.bn2.reset_parameters()
        # self.bn3.reset_parameters()

        # Initialization of Linear layers
        # torch.nn.init.xavier_uniform_(self.fc1.weight)
        # self.fc1.bias.data.fill_(0.01)
        # torch.nn.init.xavier_uniform_(self.fc2.weight)
        # self.fc2.bias.data.fill_(0.01)

        # Initialization of output Linear layer
        torch.nn.init.xavier_uniform_(self.out.weight)
        self.out.bias.data.fill_(0.01)


    def forward(self, x, edge_index, batch_index):
        # First Conv layer
        hidden = self.initial_conv(x, edge_index)
        hidden = F.relu(hidden)
        hidden = F.dropout(hidden, p=dropout_rate, training=self.training)

        # Other Conv layers
        # hidden = self.conv1(hidden, edge_index)
        # hidden = self.bn1(hidden)
        # hidden = F.dropout(hidden, p=dropout_rate, training=self.training)

        # hidden = self.conv2(hidden, edge_index)
        # hidden = self.bn2(hidden)
        # hidden = F.relu(hidden)
        # hidden = F.dropout(hidden, p=dropout_rate, training=self.training)

        # Residual Connection
        # hidden = hidden + self.initial_conv(x, edge_index)
        # hidden = self.bn3(hidden)
        # hidden = F.relu(hidden)
        # hidden = F.dropout(hidden, p=dropout_rate, training=self.training)

        # Global Pooling (stack different aggregations)
        hidden = torch.cat([gmp(hidden, batch_index), gmp(hidden, batch_index)], dim=1)

        # Additional Linear layers
        # hidden = self.fc1(hidden)
        # hidden = F.relu(hidden)
        # hidden = self.fc2(hidden)
        # hidden = F.relu(hidden)

        # Apply a final (linear) classifier.
        out = self.out(hidden)

        return out, hidden
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = GCN().to(device)
# model.init_weights()
print(model)
print("Number of parameters: ", sum(p.numel() for p in model.parameters()))




GCN(
  (initial_conv): TransformerConv(79, 32, heads=1)
  (out): Linear(in_features=64, out_features=1, bias=True)
)
Number of parameters:  10305


In [196]:
from torch_geometric.data import DataLoader
import warnings
warnings.filterwarnings("ignore")
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Root mean squared error
loss_fn = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Use GPU for training
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)


# Wrap data in a data loader
data_size = len(data_list)
NUM_GRAPHS_PER_BATCH = 128

scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, verbose=True)

In [198]:
from tqdm import tqdm

def train(loader):
    model.train()  # Установить модель в режим обучения
    total_loss = 0
    for batch in loader:
        batch = batch.to(device)  # Перевести батч на устройство GPU
        optimizer.zero_grad()
        pred, embedding = model(batch.x.float(), batch.edge_index, batch.batch)
        loss = loss_fn(pred, batch.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    return total_loss / len(loader), embedding

loss_list = []
print("Starting training...")
for epoch in tqdm(range(100)):
    loss, h = train(loader)
    loss_list.append(loss)

    scheduler.step(loss)
    if epoch % 1 == 0:
        print(f"Epoch {epoch} | Train Loss {loss:.4f}")


Starting training...


  1%|          | 1/100 [00:01<02:10,  1.32s/it]

Epoch 0 | Train Loss 1.3285


  2%|▏         | 2/100 [00:02<02:11,  1.34s/it]

Epoch 1 | Train Loss 1.3285


...

 37%|███▋      | 37/100 [00:55<01:37,  1.55s/it]

Epoch 36 | Train Loss 1.3285


 38%|███▊      | 38/100 [00:57<01:33,  1.51s/it]

Epoch 37 | Train Loss 1.3285


KeyboardInterrupt: 

In [17]:
# from torch_geometric.nn import models

# gnn_model = models.GCN(in_channels=data_list[0].num_features,dropout=0.7, hidden_channels=32, out_channels=1, num_layers=1, norm='BatchNorm',)
# gnn_model.to(device)

torch.Size([3270, 79]) torch.Size([2, 7116])

In [199]:
import pandas as pd

# Analyze the results for one batch
test_batch = next(iter(test_loader))
with torch.no_grad():
    test_batch.to(device)
    pred, embed = model(test_batch.x.float(), test_batch.edge_index, test_batch.batch)
    df = pd.DataFrame()
    df["y_real"] = test_batch.y.tolist()
    df["y_pred"] = pred.tolist()
# df["y_real"] = df["y_real"].apply(lambda row: row[0])
df["y_pred"] = df["y_pred"].apply(lambda row: row[0])
df

,y_real,y_pred
0,-2.218007,-1.045701
1,-2.062931,-1.055127
2,-1.320422,-1.050748
3,0.039781,-1.062542
4,-0.133656,-1.042516
...,...,...
1019,1.851509,-1.076200
1020,1.265848,-1.049865
1021,-0.047837,-1.056697
1022,-1.937878,-1.015981


In [200]:
df.y_pred.max() - df.y_pred.min()

0.14613312482833862

In [201]:
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# compute RMSE
rmse = np.sqrt(mean_squared_error(df['y_real'].apply(lambda x: math.pow(math.e, x) * -1), df['y_pred'].apply(lambda x: math.pow(math.e, x)  * -1)))
print(f'RMSE: {rmse}')

# Compute R2 score
r2 = r2_score(df['y_real'].apply(lambda x: math.pow(math.e, x)  * -1), df['y_pred'].apply(lambda x: math.pow(math.e, x) * -1))
print(f'R2 score: {r2}')

RMSE: 3.036261717747536
R2 score: -0.04402869496244688
